# DataFrame 마무리 & Covariance Matrix(1)
- - -

#### 1. Performing a `Map` command in DataFrame
* In order to perform a map on a dataframe, you first need to transform it into an RDD!
* Not the recommended way. Better to use built-in sparkSQL functions

<br>

#### 2. Covariance Matrix

* Calculating the mean of Sample Vectors

* Outer product of sample vectors

* Covariance Matrix
- - -

### 1. Performing a `Map` command in DataFrame

#### pyspark import & SparkContext 생성

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType

sc = SparkContext(master="local[*]")
print(sc)

# Just like using Spark requires having a SparkContext, using SQL requires an SQLContext
sqlContext = SQLContext(sc)
print(sqlContext)

<SparkContext master=local[*] appName=pyspark-shell>


In [2]:
!pip install GoogleDriveDownloader

In [3]:
#### 예제 파일 다운로드
from os.path import exists
from google_drive_downloader import GoogleDriveDownloader as gdd
import tarfile

if exists("./NY.tgz"):
    !rm -rf ./NY.tgz
if exists("./NY.parquet"):
    !rm -rf ./NY.parquet
    
gdd.download_file_from_google_drive(file_id='1hAHV6vC6FvVgrYnoN-lR-IfH488-H121',
                                   dest_path = './NY.tgz')
!tar -xzvf NY.tgz
df = sqlContext.read.load("NY.parquet")

NY.parquet/
NY.parquet/_SUCCESS
NY.parquet/part-00022-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00000-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00021-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00001-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00023-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00002-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00024-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00003-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00025-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00004-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00027-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00005-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-0000

In [4]:
# printSchema()와 sample를 이용한 데이터 확인
df.printSchema()
# sample 사용법 참조
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.sample
df.sample(False, 0.01).show(3)

root
 |-- Station: string (nullable = true)
 |-- Measurement: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Values: binary (nullable = true)
 |-- dist_coast: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)

+-----------+-----------+----+--------------------+------------------+-----------------+------------------+------------------+-----+-----------------+
|    Station|Measurement|Year|              Values|        dist_coast|         latitude|         longitude|         elevation|state|             name|
+-----------+-----------+----+--------------------+------------------+-----------------+------------------+------------------+-----+-----------------+
|USC00307659|   PRCP_s20|1917|[BF 4E B2 4E A4 4...|213.64700317382812|42.79999923706055| -74.5999984741211|249.89999389648438|   NY|SHARON SPRINGS 1N|

* DataFrame to RDD : `[DataFrame].rdd`, 각각의 요소가 `Row`인 RDD 생성

* RDD to DataFrame : `sqlContext.createDataFrame([RDD], schema)`

단, RDD에서 DataFrame으로 변환시, schema를 꼭 정의해줘야 한다!

In [5]:
some_rdd = df.rdd.takeSample(False, 1)
some_rdd

[Row(Station='USC00304102', Measurement='TOBS_s20', Year=1998, Values=bytearray(b"\xfe\xd3\x06\xd4\x0e\xd4\x18\xd4 \xd4)\xd42\xd4<\xd4D\xd4N\xd4W\xd4b\xd4k\xd4u\xd4}\xd4\x87\xd4\x8f\xd4\x96\xd4\x9d\xd4\xa6\xd4\xae\xd4\xb7\xd4\xbe\xd4\xc4\xd4\xcc\xd4\xd3\xd4\xd7\xd4\xda\xd4\xdb\xd4\xdb\xd4\xdb\xd4\xdc\xd4\xdc\xd4\xdc\xd4\xd7\xd4\xd2\xd4\xcd\xd4\xc6\xd4\xbf\xd4\xb5\xd4\xa9\xd4\x9d\xd4\x92\xd4\x87\xd4|\xd4p\xd4c\xd4U\xd4E\xd47\xd4&\xd4\x14\xd4\x05\xd4\xeb\xd3\xcc\xd3\xaa\xd3\x85\xd3`\xd3:\xd3\x14\xd3\xee\xd2\xc7\xd2\x9c\xd2t\xd2H\xd2\x1c\xd2\xf1\xd1\xc5\xd1\x98\xd1i\xd1;\xd1\t\xd1\xd4\xd0\xa0\xd0l\xd06\xd0\xfc\xcf\x86\xcf\x14\xcf\x9e\xce-\xce\xbb\xcdE\xcd\xcf\xcc[\xcc\xc5\xcb\xc3\xca\xb7\xc9\xb7\xc8k\xc7R\xc5\xa3\xc28\xbds9ZA\xd2D\xf0F\x80H\x8eI\x8fJ\x8fKIL\xcaLKM\xceMLN\xcdNMO\xcfO\'PgP\xa6P\xe7P\'QhQ\xa8Q\xe8Q\'ReR\xa3R\xe6R\'SjS\xa7S\xe4S\x11T1TNTjT\x86T\xa2T\xbdT\xd9T\xf4T\x10U*UBU[UsU\x8bU\xa2U\xbaU\xd3U\xebU\x05V\x1eV8VRVlV\x84V\x9dV\xb5V\xceV\xe7V\x00W\x17W-WBWYWqW\x87W\x9eW\xb4W\x

* Lec 6의 예제

    1. 주어진 DataFrame에서 `Year가 1900 미만인 경우 '19th'`, `2000 미만인 경우 '20th'`, `2010 미만인 경우 '21st'`, `모두 아닐 경우 'possibly_bad_data'로 값을 치환`하여라

 여기서 `map`에 들어가는 `input`이 무엇인지 반드시 숙지하여야 한다.
    


In [6]:
def find_century(row):
    if row.Year < 1900:
        return "19th"
    elif row.Year < 2000:
        return "20th"
    elif row.Year < 2010:
        return "21st"
    else:
        return "possibly_bad_data"
    
df.rdd.map(find_century).take(5)

['20th', '20th', '20th', '20th', '20th']

* Lec 6의 예제

    2. 주어진 DataFrame에서 각각의 요소 중 `longitude`와 `latitude`를 추출하여 (longitude, latitude)의 형태로 값을 출력하여라.

In [7]:
df.rdd.map(lambda row : (row.longitude, row.latitude)).take(5)

[(-77.71330261230469, 42.57080078125),
 (-77.71330261230469, 42.57080078125),
 (-77.71330261230469, 42.57080078125),
 (-77.71330261230469, 42.57080078125),
 (-77.71330261230469, 42.57080078125)]

### Excercise 1 - RDD function in DataFrame (60 point)

- - -
task 별로 ``위에서 사용한 DataFrame(df)``에서 ``RDD``로 변환 후 ``RDD function``을 적용하여 해결합니다

(task 당 20 point)

---

**task**

* 1 : ``df``에서 ``name``별 가장 최근 ``Year``를 **map과 reduce, 또는 reduceByKey 등**을 활용하여 구한 후, take(10)을 통하여 출력합니다.(20 point)

<br>

* 2 : ``df``에서 ``Year``가 ``2000 이상``인 결과에 대해, ``name``별 ``Year``, ``dist_coast``, ``elevation``의 평균을 구하고 ``name``를 기준으로 정렬(Z->A)한 후, take(5)을 통하여 출력합니다.(20 point)

<br>

* 3 : ``df``에서 ``Measurement`` 별 `Values`의 1일부터 10일까지(``np.frombuffer(row.Values[:20], dtype='float16')`` 또는 ``np.frombuffer(row.Values, dtype = 'float16')[:10])의 합이 가장 큰 ``Year``와 ``그 값을 구한 후``, ``Measurement``를 기준으로 정렬(A->Z)합니다. 마지막으로 collect를 하여 출력합니다.(20 point) 

- - -

**task**

* 1 : ``df``에서 ``name``별 가장 최근 ``Year``를 **map과 reduce, 또는 reduceByKey 등**을 활용하여 구한 후, take(10)을 통하여 출력합니다.(20 point)

**★ DataFrame이 아닌 RDD로 작업할 것**

```
#task1 output
[('DANSVILLE MUNI AP', 2013),
 ('BRIDGEHAMPTON', 2013),
 ('MIDDLETOWN 2 NW', 2011),
 ('BERLIN 5 S', 2000),
 ('ELMIRA CORNING RGNL AP', 2013),
 ('UNADILLA 2 N', 2013),
 ('SUFFERN 2 E', 1955),
 ('ROXBURY', 1972),
 ('LOWVILLE', 2013),
 ('GABRIELS', 1978)]

```

In [8]:
# 1-1 답 작성
df.rdd.map(lambda row :(row.name, row.Year)).reduceByKey(lambda x,y : x if x > y else y).take(10)
# df의 데이터를 rdd의 row의 튜플로 받아 reduceByKey를 하여 key 별로 value 값을 비교하여 큰 값을 추출.

[('CANTON 4 SE', 2013),
 ('VICTOR 2NW', 2012),
 ('UNADILLA 2 N', 2013),
 ('LOWVILLE', 2013),
 ('ADAMS CTR', 1950),
 ('HONEOYE', 2013),
 ('EAST SIDNEY', 2013),
 ('NEW YORK BENSONHURST', 1953),
 ('DELANSON 2NE', 2013),
 ('CANAJOHARIE', 1976)]

**task**

* 2 : ``df``에서 ``Year``가 ``2000 이상``인 결과에 대해, ``name``별 ``Year``, ``dist_coast``, ``elevation``의 평균을 구하고 ``name``를 기준으로 정렬(Z->A)한 후, take(10)을 통하여 출력합니다.(20 point)

**★ DataFrame이 아닌 RDD로 작업할 것**

```
# task2 output
[('YOUNGSTOWN 2 NE', (2008.5, 476.80999755859375, 85.30000305175781)),
 ('YORKTOWN HEIGHTS 1W',
  (2006.421686746988, 28.945999145507812, 204.1999969482422)),
 ('WINDHAM 3 E', (2004.8387096774193, 147.28700256347656, 512.0999755859375)),
 ('WILLSBORO 1 N',
  (2005.360655737705, 255.24200439453125, 54.900001525878906)),
 ('WHITNEY POINT DAM', (2009.2826086956522, 235.35800170898438, 317.0))]
 ```

In [9]:
# 1-2 답 작성
df.rdd.map(lambda row : (row.name,(row.Year, row.dist_coast, row.elevation,1))).filter(lambda x : x[1][0] > 2000).reduceByKey(lambda x,y : tuple([(x[i] + y[i]) for i in range(len(x))])).map(lambda x:(x[0],tuple([x[1][0]/x[1][3], x[1][1]/x[1][3], x[1][2]/x[1][3]]))).sortByKey(False).take(10)
# df rdd를 map하여 row로 받아 name key를 가지고 value로 Year와 dist_coast, elevaion,1을 가지는 새로운 rdd를 생성 후 
#filter를 통해 새로 변형된 df rdd의 1번째의 0번째 인덱스가 2000 인상인 값만 추출 한 후, 
#reduceByKey를 통해 키 값을 기준으로 value들을 다 더한 후 다시 map을 사용해 각 인자들을 하나씩 접근하여 인덱싱 하여 평균을 구하고, 
#sortBykey를 flase로 지정하여 내림차순으로 정렬한다. 

[('YOUNGSTOWN 2 NE', (2008.5, 476.80999755859375, 85.30000305175781)),
 ('YORKTOWN HEIGHTS 1W',
  (2006.922077922078, 28.945999145507812, 204.1999969482422)),
 ('WINDHAM 3 E', (2005.357142857143, 147.28700256347656, 512.0999755859375)),
 ('WILLSBORO 1 N',
  (2005.8392857142858, 255.24200439453125, 54.900001525878906)),
 ('WHITNEY POINT DAM', (2009.2826086956522, 235.35800170898438, 317.0)),
 ('WHITEHALL', (2006.922077922078, 220.0189971923828, 36.29999923706055)),
 ('WESTHAMPTN GABRESKI AP',
  (2006.7317073170732, 4.3188700675964355, 20.399999618530273)),
 ('WESTFIELD 2 SSE', (2002.0, 418.3210144042969, 215.5)),
 ('WESTCHESTER CO AP', (2006.5348837209303, 7.586299896240234, 115.5)),
 ('WEST POINT', (2006.5, 47.6151008605957, 97.5))]

**task**

* 3 : ``df``에서 ``Measurement`` 별 `Values`의 1일부터 10일까지(``np.frombuffer(row.Values[:20], dtype='float16')`` 또는 ``np.frombuffer(row.Values, dtype = 'float16')[:10])의 합이 가장 큰 ``Year``와 ``그 값을 구한 후``, ``Measurement``를 기준으로 정렬(A->Z)합니다. 마지막으로 collect를 하여 출력합니다.(20 point) 

<br>

**★ DataFrame이 아닌 RDD로 작업할 것**

★★여기서 ``Values``는 ``bytearray`` type입니다. ★★

``numpy``의 **frombuffer**를 이용하여 ``float16``으로 바꿉니다.
자세한 사용법은 [여기](https://docs.scipy.org/doc/numpy/reference/generated/numpy.frombuffer.html)를 참고합니다

★★또한, ``numpy``의 ``nansum``을 이용하여 값이 **nan**이 아닌 Values의 합을 구합니다.★★

```
# task3 output
[('PRCP', (12824.0, 1946)),
 ('PRCP_s20', (4264.0, 1983)),
 ('SNOW', (2912.0, 1954)),
 ('SNOW_s20', (4140.0, 1895)),
 ('SNWD', (16590.0, 1970)),
 ('SNWD_s20', (12320.0, 1976)),
 ('TMAX', (2216.0, 1897)),
 ('TMAX_s20', (2050.0, 1897)),
 ('TMIN', (623.0, 2007)),
 ('TMIN_s20', (21360.0, 1987)),
 ('TOBS', (1000.0, 1998)),
 ('TOBS_s20', (969.5, 1992))]
 ```

In [10]:
import numpy as np
df.rdd.map(lambda row : (row.Measurement,(np.nansum(np.frombuffer(row.Values[:20], dtype='float16')),row.Year))).reduceByKey(lambda x,y : x if x > y else y).sortByKey().collect()
# df을 rdd로 사용하며 map을 사용하여 새로운 rdd형태를 만드는데 Measurement를 key값으로 value는 Values와 Year이다.
# 여기서 Value의 데이터 타입이 bytearry이기 때문에 numpy의 frombuffer를 사용하여 float16으로 변환한다.
# 여기서 np.nansum을 사용하여 Values의 값이 nan을 제외하고 합한다.
# 그런 후 reduceByKey를 사용하여 합의 값이 가장 큰 값을 Key를 기준으로 뽑아 낸 후 sortByKey를 적용하여 오름차순으로 정렬한다.

[('PRCP', (12824.0, 1946)),
 ('PRCP_s20', (4264.0, 1983)),
 ('SNOW', (2912.0, 1954)),
 ('SNOW_s20', (4140.0, 1895)),
 ('SNWD', (16590.0, 1970)),
 ('SNWD_s20', (12320.0, 1976)),
 ('TMAX', (2216.0, 1897)),
 ('TMAX_s20', (2050.0, 1897)),
 ('TMIN', (623.0, 2007)),
 ('TMIN_s20', (21360.0, 1987)),
 ('TOBS', (1000.0, 1998)),
 ('TOBS_s20', (969.5, 1992))]

### 2. Spark와 Numpy를 사용하여 Covariance Matrix 구해보자!

#### (1) numpy 기초 
아래 참고자료를 활용하여 numpy 기초 학습

* 참고자료 
  * [참고자료 1](http://taewan.kim/post/numpy_cheat_sheet/)
  * [참고자료 2](https://docs.scipy.org/doc/numpy/user/quickstart.html)
  * [참고자료 3](https://scipy-lectures.org/intro/numpy/array_object.html#what-are-numpy-and-numpy-arrays)
  * [참고자료 4](https://doorbw.tistory.com/171)
  * [참고자료 5](https://datascienceschool.net/view-notebook/17608f897087478bbeac096438c716f6/)

* 위 자료에서중에서도 
  * ndarray 생성법
  * vector, matrix 연산
  * 인덱싱 (slicing)
  * 행렬 합치기 (vstack, dstack, hstack)
  * sum, mean
  * np.nan 자료형
  * reshape
  * matmul


#### (2)  Calculating the mean of Sample Vectors 
다음 벡터들을 샘플 벡터로 가정합니다.
* $n$ 은 샘플의 수 이고
* $d$는 각 데이터 벡터의 길이 입니다 (예를 들어서 날씨데이터의 경우 $d=365$)
$$
\mathbf{x}_i = \left[\begin{array}{cccc}
x_{i1} & x_{i2}& \ldots & x_{id}, 
\end{array}\right], \quad i=1,\ldots, n
$$
Sample vector 들의 mean (평균) 벡터 $\bar{\mathbf{x}}$는 다음과 같이 구합니다 
$$
\bar{\mathbf{x}} = \frac{1}{n}\sum_{i=1}^n \mathbf{x}_i
$$


In [0]:
# 평균 벡터 구하는 문제 (sum, mean 활용 둘다 해도 괜찮습니다))

In [11]:
# in python
import numpy as np

sample1 = np.array([1,2,3])
sample2 = np.array([4,5,6])
sample3 = np.array([7,8,9])
print("각 sample의 값 : ", sample1, sample2, sample3)
print("sample vector sum : ", sample1 + sample2 + sample3)
print("sample vector mean : ", (sample1 + sample2 + sample3) / 3)

각 sample의 값 :  [1 2 3] [4 5 6] [7 8 9]
sample vector sum :  [12 15 18]
sample vector mean :  [4. 5. 6.]


In [12]:
# in spark
vector_list = sc.parallelize([np.array([1,2,3]),np.array([4,5,6]),np.array([7,8,9])])
print("각 sample의 값 : ", vector_list.collect())
print("sample vector sum : ", vector_list.reduce(lambda ndarr1, ndarr2 : ndarr1 + ndarr2))
print("sample vector mean : ",
      vector_list.reduce(lambda ndarr1, ndarr2 : ndarr1 + ndarr2) / vector_list.count())

각 sample의 값 :  [array([1, 2, 3]), array([4, 5, 6]), array([7, 8, 9])]
sample vector sum :  [12 15 18]
sample vector mean :  [4. 5. 6.]


### Excercise 2 - Calculating the mean of Sample Vectors (40 point)

- - -
 
다음 데이터에 대하여 다음 과제를 수행하세요.

- regular.csv : KBO에서 활약한 타자들의 역대 정규시즌 성적을 포함하여 몸무게, 키 ,생년월일 등의 기본정보

**위의 두 데이터는 모두 `,`로 구분되어 있습니다.**

 - **데이터의 자세한 설명은 다음의 링크를 참조해주세요.([여기를 눌러서 12. 데이터 설명 참고](https://dacon.io/cpt6/62885))**
 - 또한 regular.csv를 직접 열어서 데이터가 어떻게 저장되어 있는지 확인해주세요.

- - -

**task**

- 1. filter를 사용하여 팀 이름이 ``두산``인 선수에 대해, ``(batter_id, np.array([G,R,H,RBI,BB]))``의 형태로 Key/Value RDD를 생성합니다. (20 point)

    1. G R H RBI BB의 경우 초기 설정값이 ``stirng``.  이 값들을 ``float64``으로 변경할 것. ex) ``np.array([1,2,3], dtype = 'float64')``
    2. ★ 각 값이 `' '`일 경우, 0 으로 변경할 것. 
    3. ``map``에서 바로 적용 또는 그러한 함수를 작성
    

<br>

- 2. (20 point)
    1. ``reduceByKey``를 사용하여 ``batter_id``(Key)가 동일한 선수의 ``G, R, H, RBI, BB``(Value)를 각각 더해준 후 ``batter_id``(Key)를 기준으로 ``sortByKey``를 적용합니다. 그 후, ``map``을 사용하여 ``G, R, H, RBI, BB``(Value)만 선택 후 새로운 RDD로 만듭니다.
    2. 위에서 생성된 RDD에 대해, 중복되지 않는 sample의 수를 ``count``를 이용하여 구합니다.
    3. ``reduce``를 이용하여 ``G, R, H, RBI, BB``(Value)를 모두 더한 후 위에서 구한 sample의 수(`count`)로 나누어서 sample vector의 평균을 구합니다.



---

In [13]:
import urllib.request
import re

f = urllib.request.urlretrieve ("https://docs.google.com/uc?export=download&id=1b_L-rJYJC9Oqga0fQ2zh2M763CTM8jzR", "regular.csv")
regular = sc.textFile("./regular.csv").map(lambda x : x.split(","))

**task**

- 1. filter를 사용하여 팀 이름이 ``두산``인 선수에 대해, ``(batter_id, np.array([G,R,H,RBI,BB]))``의 형태로 Key/Value RDD를 생성합니다. (20 point)

    1. G R H RBI BB의 경우 초기 설정값이 ``stirng``.  이 값들을 ``float64``으로 변경할 것. ex) ``np.array([1,2,3], dtype = 'float64')``
    2. ★ 각 값이 `' '`일 경우, 0 으로 변경할 것. 
    3. ``map``에서 바로 적용 또는 그러한 함수를 작성
    
```
#output
[(7, array([1., 0., 0., 0., 0.])),
 (7, array([64., 16., 21., 11.,  6.])),
 (7, array([93., 30., 40., 25., 16.])),
 (7, array([6., 3., 3., 3., 2.])),
 (7, array([40., 14., 17.,  5., 10.])),
 (7, array([48., 12., 24., 10., 18.])),
 (17, array([16.,  1.,  1.,  1.,  0.])),
 (17, array([32.,  2.,  3.,  0.,  0.])),
 (17, array([16.,  2.,  2.,  0.,  0.])),
 (17, array([116.,  38.,  85.,  29.,  24.]))]
 ```

In [80]:
import numpy as np
task1 = regular.filter(lambda x : x[3] == '두산').map(lambda x : (x[0],(np.array([x[5],x[7],x[8],x[13],x[16]],dtype = 'float64'))))
task1.take(10)
# regular 데이터셋에서 filter를 사용하여 데이터 column의 3번째 인자인 team명이 '두산'인 값만 추출한다.
# 그런 후 map을 사용하여 데이터셋의 0번째(batter_id)를 key로 가지고, 5번째(G),7번째(R),8번째(H),13번째(RBI),16번째(BB)를 value로 가지는 RDD를 생성한다.
# 여기서 numpy의 array의 형태로 출력는데 dataType이 string인 값을 float64로 변경한다.

[('7', array([1., 0., 0., 0., 0.])),
 ('7', array([64., 16., 21., 11.,  6.])),
 ('7', array([93., 30., 40., 25., 16.])),
 ('7', array([6., 3., 3., 3., 2.])),
 ('7', array([40., 14., 17.,  5., 10.])),
 ('7', array([48., 12., 24., 10., 18.])),
 ('17', array([16.,  1.,  1.,  1.,  0.])),
 ('17', array([32.,  2.,  3.,  0.,  0.])),
 ('17', array([16.,  2.,  2.,  0.,  0.])),
 ('17', array([116.,  38.,  85.,  29.,  24.]))]

**task**

- 2. (20 point)
    1. ``reduceByKey``를 사용하여 ``batter_id``(Key)가 동일한 선수의 ``G, R, H, RBI, BB``(Value)를 각각 더해준 후 ``batter_id``(Key)를 기준으로 ``sortByKey``를 적용합니다. 그 후, ``map``을 사용하여 ``G, R, H, RBI, BB``(Value)만 선택 후 새로운 RDD로 만듭니다.
    2. 위에서 생성된 RDD에 대해, 중복되지 않는 sample의 수를 ``count``를 이용하여 구합니다.
    3. ``reduce``를 이용하여 ``G, R, H, RBI, BB``(Value)를 모두 더한 후 위에서 구한 sample의 수(`count`)로 나누어서 sample vector의 평균을 구합니다.

```
# output
[G, R, H, RBI, BB] mean :  [427.8846  186.32692 344.05768 178.0577  129.48077]
```

In [108]:
task2 = task1.reduceByKey(lambda x,y : x + y).sortByKey().map(lambda x : x[1])
task2_mean_vector = task2.reduce(lambda x,y : x+y) / task2.count() # 52
print("[G, R, H, RBI, BB] mean : ", task2_mean_vector)

# 위의 task에서 생성한 rdd를 reduceByKey를 적용하여 같은 key를 가진 value들을 합하고, sortByKey를 적용하여 오름차순으로 정렬 한 후,
# map을 사용하여 기존 RDD 리스트의 1번째 인자들로만 구성된 새로운 RDD를 생성한후 task2에 저장한다.
# task2에 count()함수를 적용시켜보면 52가 나오고,새로 생성된 task2 RDD를 reduce하여 값들을 모두 합한 후 count()한 값으로 나누어 준다.


[G, R, H, RBI, BB] mean :  [427.88461538 186.32692308 344.05769231 178.05769231 129.48076923]


# 기말고사가 다가온다.ㅋ